In [10]:
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   


In [11]:
df = pdr.get_data_yahoo("AMAT", start="2010-01-01",end="2021-11-26")
sp = "^GSPC"
spdf = pdr.get_data_yahoo(sp, start="2010-01-01",end="2021-11-26")
sortLogic = {
    'Close': 'last',
    'Volume': 'sum'
}
dfSorted = df.resample("W-FRI").agg(sortLogic)
spDfSorted = spdf.resample("W-FRI").agg(sortLogic)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)

In [13]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['Close'],name='Price'),secondary_y=True)
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['30WMA'],name="30WMA"),secondary_y=True)
fig.add_trace(go.Bar(x=dfSorted.index, y=dfSorted['Volume'],name="Volume"),secondary_y=False)
fig.show()

In [14]:
deltaX = 10.4
dfSorted['30WMASlope'] = dfSorted['30WMA'].diff()*500/(dfSorted['Close'].rolling(window=104).max()-dfSorted['Close'].rolling(window=104).min())/deltaX

In [15]:
dfSorted['P4WH'] = dfSorted['Close'].rolling(window=4).max()
dfSorted['P4WL'] = dfSorted['Close'].rolling(window=4).min()

In [16]:
dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change()
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].Close/df.iloc[index-30].Close
dfSorted['Percent'] = dfSorted.apply(lambda x: product(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
spDfSorted['Percent'] = dfSorted.apply(lambda x: product(spDfSorted,spDfSorted.index.get_loc(x.name)),axis=1)
dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
fullPrint(dfSorted['RS'])

Date
2010-01-08    0.000000
2010-01-15    0.000000
2010-01-22    0.000000
2010-01-29    0.000000
2010-02-05    0.000000
2010-02-12    0.000000
2010-02-19    0.000000
2010-02-26    0.000000
2010-03-05    0.000000
2010-03-12    0.000000
2010-03-19    0.000000
2010-03-26    0.000000
2010-04-02    0.000000
2010-04-09    0.000000
2010-04-16    0.000000
2010-04-23    0.000000
2010-04-30    0.000000
2010-05-07    0.000000
2010-05-14    0.000000
2010-05-21    0.000000
2010-05-28    0.000000
2010-06-04    0.000000
2010-06-11    0.000000
2010-06-18    0.000000
2010-06-25    0.000000
2010-07-02    0.000000
2010-07-09    0.000000
2010-07-16    0.000000
2010-07-23    0.000000
2010-07-30    0.000000
2010-08-06   -0.165870
2010-08-13   -0.136472
2010-08-20   -0.103549
2010-08-27   -0.113690
2010-09-03   -0.138149
2010-09-10   -0.175195
2010-09-17   -0.133204
2010-09-24   -0.089837
2010-10-01   -0.053814
2010-10-08   -0.056872
2010-10-15   -0.051674
2010-10-22   -0.095891
2010-10-29   -0.079286
2010-1

In [17]:
def checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL):
    stageOneIndicator = 0
    falseReason = ""
    if volumePerc<0.7:
        stageOneIndicator+=1
    else:
        falseReason += "volume "
    if slope<=0.5 and slope>=-0.3:
        stageOneIndicator+=1
    else:
        falseReason += "slope "
    if P4WH<P4WL*1.15:
        stageOneIndicator+=1
    else:
        falseReason += "range "
    if price <= WMA*1.1 and price>= WMA*0.9:
        stageOneIndicator+=1
    else:
        falseReason += "price "
    if stageOneIndicator>=3:
        return True
    return "False " + falseReason
def checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage):
    if volumePerc < 0.7:
        if prevStage != "Stage 2":
            #print(prevStage)
            return "Volume"
    if RS < 0.1 and prevStage != "Stage 2":
        return "RS"
    if RS < 0 and prevStage == "Stage 2":
        return "RS"
    if slope < 0.5 and prevStage != "Stage 2":
        return "Slope"
    if slope < 0.3 and prevStage == "Stage 2":
        return "Slope"
    if price < WMA*1.1 and prevStage != "Stage 2":
        return "Price"
    if price < WMA*0.95 and prevStage == "Stage 2":
        return "Price"
    return "Clear"
def checkStage(price,volumePerc, RS, slope, WMA,P4WH,P4WL,prevStage):
    stage1Check = checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL)
    stage2Check = checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage)
    if stage2Check == "Clear":
        return "Stage 2"
    if stage1Check == True:
        return "Stage 1 " + stage2Check 
    return stage2Check       

In [18]:
dfSorted = dfSorted.dropna()
dfSorted['Stage'] = ""
#dfSorted['Stage'] = dfSorted.apply(lambda x: checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.loc[index]['Stage'].shift()),axis=1)
for index, element in dfSorted.iterrows():
    if dfSorted.index.get_loc(index) == 0:
        continue
    dfSorted.iloc[dfSorted.index.get_loc(index), dfSorted.columns.get_loc('Stage')] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Stage'])
    #dfSorted.loc[index]['Stage'] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Stage'])
    #dfSorted.loc[index]['Stage'] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.shift(1).loc[index]['Stage'])
fullPrint(dfSorted[['Close','Stage']])

                 Close           Stage
Date                                  
2011-12-30   10.710000                
2012-01-06   11.010000  Stage 1 Volume
2012-01-13   11.500000      Stage 1 RS
2012-01-20   12.470000          Volume
2012-01-27   12.230000  Stage 1 Volume
2012-02-03   12.790000          Volume
2012-02-10   12.950000          Volume
2012-02-17   12.990000          Volume
2012-02-24   12.630000  Stage 1 Volume
2012-03-02   12.220000  Stage 1 Volume
2012-03-09   12.370000  Stage 1 Volume
2012-03-16   12.820000  Stage 1 Volume
2012-03-23   12.690000  Stage 1 Volume
2012-03-30   12.450000  Stage 1 Volume
2012-04-06   12.030000  Stage 1 Volume
2012-04-13   11.800000  Stage 1 Volume
2012-04-20   11.770000  Stage 1 Volume
2012-04-27   12.030000  Stage 1 Volume
2012-05-04   11.280000  Stage 1 Volume
2012-05-11   10.980000  Stage 1 Volume
2012-05-18   10.360000          Volume
2012-05-25   10.540000          Volume
2012-06-01   10.010000          Volume
2012-06-08   10.760000  S